In [3]:
import optuna

import sklearn.datasets
import xgboost as xgb


def objective(trial):
    train_x, train_y = sklearn.datasets.load_breast_cancer(return_X_y=True)
    dtrain = xgb.DMatrix(train_x, label=train_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "test-auc")
    history = xgb.cv(param, dtrain, num_boost_round=100, callbacks=[pruning_callback])

    mean_auc = history["test-auc-mean"].values[-1]
    return mean_auc


if __name__ == "__main__":
    pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
    study = optuna.create_study(pruner=pruner, direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-05-29 18:33:00,175] A new study created in memory with name: no-name-2364dd95-5c47-4579-b73e-959f77497c25
[W 2024-05-29 18:33:00,206] Trial 0 failed with parameters: {'booster': 'gbtree', 'lambda': 5.171775688696952e-05, 'alpha': 1.240042393827509e-07, 'max_depth': 7, 'eta': 4.0412946344311053e-07, 'gamma': 0.012655517156345403, 'grow_policy': 'depthwise'} because of the following error: ModuleNotFoundError("Optuna's integration modules for third-party libraries have started migrating from Optuna itself to a package called `optuna-integration`. The module you are trying to use has already been migrated to `optuna-integration`. Please install the package by running `pip install optuna-integration`.").
Traceback (most recent call last):
  File "/Users/natalie/anaconda3/lib/python3.8/site-packages/optuna/integration/__init__.py", line 141, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "/Users/natalie/anaconda3/lib/python3.8/importlib/__

ModuleNotFoundError: Optuna's integration modules for third-party libraries have started migrating from Optuna itself to a package called `optuna-integration`. The module you are trying to use has already been migrated to `optuna-integration`. Please install the package by running `pip install optuna-integration`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, roc_auc_score, f1_score, accuracy_score,confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPClassifier
   

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')

/Users/natalie/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
test = pd.read_parquet('/Users/natalie/Desktop/DS Thesis/Code/data/test.parquet')
train = pd.read_parquet('/Users/natalie/Desktop/DS Thesis/Code/data/train.parquet')

In [5]:
TARGET ='churn_user'
CATEGORICAL_FEATURES  = ['os_name', 'age_group','gender', 'country', 'region', 'province_type',
                         'province']
DATETIME_FEATURES  = ['first_date', 'lastest_active_day']
SEARCH_CC_FEATURES = [ 'clicks', 'search_volume', 'dating_search', 'videoclip_search', 'technical_search', 'housekeeping_family_search', 'marketing_search', 'other_search']
SEARCH_GG_FEATURES = [ 'serp_click', 'search_volume_gg', 'search_clicks_gg', 'other_search_gg','housekeeping_family_search_gg','videoclip_search_gg', 'dating_search_gg', 'marketing_search_gg', 'technical_search_gg']
ACTIVE_FEATURES = ['active_day', 'life_time',  'not_active_day', 'total_active_time']
ADS_FEATURES =  ['ads_impression', 'ads_click', 'ads_revenue']
OTHERS_FEATURES =[                      'newtab_count', 'download_count', 'pip_count', 'sidebar_count', 'incognito_count', 'signin_count', 'youtube_count',
                    'work_count', 'social_count', 'news_count', 'entertainment_count', 'ecommerce_count']
NUMERICAL_FEATURES = SEARCH_CC_FEATURES + SEARCH_GG_FEATURES + ACTIVE_FEATURES + ADS_FEATURES + OTHERS_FEATURES

MODEL_NAMES = ['log_reg', 'randomforest','lightgbm', 'xgboost', 'mlp']


In [6]:
def find_best_threshold_f1_score(train_labels, oofs, average='macro'):
    scores = []
    thresholds = []
    best_score = 0
    best_threshold = 0
    for threshold in np.arange(0.1, 0.9, 0.01):
        print(f'{threshold:.02f}, ', end='')
        preds = (oofs > threshold).astype('int')
        m = f1_score(train_labels, preds, average=average)
        scores.append(m)
        thresholds.append(threshold)
        if m > best_score:
            best_score = m
            best_threshold = threshold
    return best_threshold, best_score


In [7]:
def flat_feature_engineer(df):
    temp = pd.DataFrame()
    temp['OTHERS_FEATURES_SUM'] = df[OTHERS_FEATURES].sum(axis=1)
    temp['OTHERS_FEATURES_MIN'] = df[OTHERS_FEATURES].min(axis=1)
    temp['OTHERS_FEATURES_MAX'] = df[OTHERS_FEATURES].max(axis=1)
    temp['OTHERS_FEATURES_MEAN'] = df[OTHERS_FEATURES].mean(axis=1)
    temp['ADS_CTR'] = df['ads_click'] / df['ads_impression']
    temp['SEARCH_CC_FEATURES_SUM'] = df[SEARCH_CC_FEATURES[1:]].sum(axis=1)
    temp['SEARCH_CC_FEATURES_MIN'] = df[SEARCH_CC_FEATURES[1:]].min(axis=1)
    temp['SEARCH_CC_FEATURES_MAX'] = df[SEARCH_CC_FEATURES[1:]].max(axis=1)
    temp['SEARCH_CC_FEATURES_MEAN'] = df[SEARCH_CC_FEATURES[1:]].mean(axis=1)
    temp['SEARCH_GG_FEATURES_SUM'] = df[SEARCH_GG_FEATURES[1:]].sum(axis=1)
    temp['SEARCH_GG_FEATURES_MIN'] = df[SEARCH_GG_FEATURES[1:]].min(axis=1)
    temp['SEARCH_GG_FEATURES_MAX'] = df[SEARCH_GG_FEATURES[1:]].max(axis=1)
    temp['SEARCH_GG_FEATURES_MEAN'] = df[SEARCH_GG_FEATURES[1:]].mean(axis=1)
    temp['not_active_day_per_active_day'] = df['not_active_day'] / df['active_day']
    temp['life_time_per_active_day'] = df['life_time'] / df['active_day']
    return temp, list(temp.columns)

def fillna(df):
    df['total_active_time'] = df['total_active_time'].fillna(0)
    df['ads_impression'] = df['ads_impression'].fillna(0)
    df['ads_click'] = df['ads_click'].fillna(0)
    df['ads_revenue'] = df['ads_revenue'].fillna(0)
    df['clicks'] = df['clicks'].fillna(0)
    for c in OTHERS_FEATURES:
        df[c] = df[c].fillna(0)
    return df

def process_data(df,oh_encoder=None, robust_scaler=None,agg_features=None):
    if not oh_encoder:
        print("fit train OneHotEncoder")
        oh_encoder = OneHotEncoder()
        oh_encoder.fit(df[CATEGORICAL_FEATURES])
    else:
        print("loadd onehot encoder")
    if not robust_scaler:
        print("fit train RobustScaler")
        robust_scaler = RobustScaler()
        robust_scaler.fit(df[NUMERICAL_FEATURES])
    else:
        print("loadd robust scaler")
    df_cat = pd.DataFrame(oh_encoder.transform(df[CATEGORICAL_FEATURES]).toarray())
    new_cat_cols = oh_encoder.get_feature_names_out(CATEGORICAL_FEATURES)
    df_cat.columns = new_cat_cols
    df_num = pd.DataFrame(robust_scaler.transform(df[NUMERICAL_FEATURES]))
    df_num.columns = NUMERICAL_FEATURES
    new_df = pd.concat([df_cat.reset_index(drop=True), df_num.reset_index(drop=True)], axis=1)
    new_df = fillna(new_df)
    return new_df, oh_encoder, robust_scaler

In [25]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

def cross_validate_tune_hyperparameters(train, params, model_name='randomforest', callbacks=None):
    auc_scores = []
    for i, (train_index, valid_index) in enumerate(kfold.split(train, train[TARGET])):
        print(f"===========fold {i}================")
        X_train, oh_encoder, robust_scaler = process_data(train.iloc[train_index])
        X_valid, _, _  = process_data(train.iloc[valid_index], oh_encoder,robust_scaler)
        y_train = train.iloc[train_index][TARGET].values
        y_valid = train.iloc[valid_index][TARGET].values
        print(X_train.shape)
        if model_name == "rf":
            print("Random Forest--------------")
            rf_model = RandomForestClassifier(**params)
            rf_model.fit(X_train, y_train)
            rf_y_pred_proba = rf_model.predict_proba(X_valid)[:,1]
            best_threshold, best_score = find_best_threshold_f1_score(y_valid, rf_y_pred_proba)
            print(f"\n best_threshold {best_threshold} best_score {best_score}")

            auc_scores.append(roc_auc_score(y_valid, rf_y_pred_proba))
        elif model_name == 'lgb':
    #     models.append(model)
            print("LGBModel--------------")
            lgb_model = LGBMClassifier(**params)
            # callbacks = [lgb.early_stopping(200, verbose=50), lgb.log_evaluation(period=50)]
            lgb_model.fit(X_train, y_train,
                    eval_set=[(X_train, y_train), (X_valid, y_valid)],
                    eval_metric=["logloss", "auc"])

            lgb_y_pred_proba = lgb_model.predict_proba(X_valid)[:,1]
            auc_scores.append(roc_auc_score(y_valid, lgb_y_pred_proba))
    #     models.append(model)
        # display(pd.DataFrame({'score': lgb_model.feature_importances_, 'feature': lgb_model.feature_name_}).sort_values('score',ascending=False))
        elif model_name =='xgb':
            print("XGBoost--------------")
            xgb_model = XGBClassifier(**params)
            xgb_model.fit(X_train, y_train,
                    eval_set=[(X_train, y_train), (X_valid, y_valid)],
                    verbose=50, callbacks=callbacks)
            xgb_y_pred_proba = xgb_model.predict_proba(X_valid)[:,1]
            auc_scores.append(roc_auc_score(y_valid, xgb_y_pred_proba))
        elif model_name == 'mlp':        
            print("MLP------------------")
            mlp_model = MLPClassifier(**params)
            mlp_model.fit(X_train, y_train)
            mlp_y_pred_proba = mlp_model.predict_proba(X_valid)[:,1]
            auc_scores.append(roc_auc_score(y_valid, mlp_y_pred_proba))
    return np.mean(auc_scores)

In [51]:
import optuna

import sklearn.datasets
import xgboost as xgb


def objective(trial,train=None, model_name='rf'):

    lgb_param = {
            "metric": "auc",
            "n_estimators": 200,
            'learning_rate':0.03,
            "learning_rate":trial.suggest_float("learning_rate",0.01,0.1),
            "max_depth":trial.suggest_int("max_depth",1, 9),
            'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.1),
            'subsample':trial.suggest_discrete_uniform('subsample',0.1,1,0.1),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            "random_state":42,
            'device':'gpu',
        #     "class_weight": "balanced"
        }
    xgb_param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "n_estimators":200,
        "learning_rate":trial.suggest_float("learning_rate",0.01,0.1),
        "max_depth":trial.suggest_int("max_depth",1, 9),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.1),
        'subsample':trial.suggest_discrete_uniform('subsample',0.1,1,0.1),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        'tree_method':'gpu_hist',
        'seed':42,

    }
    rf_params = {
        'n_estimators':200,
        "max_depth":trial.suggest_int("max_depth",1, 9),
        'max_features':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.1),
    }
    mlp_params = {'hidden_layer_sizes':(trial.suggest_int("hidden_layer_sizes",2, 5),125), 'random_state':42, 'max_iter':100}
    if model_name == 'rf':
        mean_auc = cross_validate_tune_hyperparameters( train=train, params=rf_params, model_name=model_name)

    elif model_name == 'xgb':
        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_1-auc")
        mean_auc = cross_validate_tune_hyperparameters( train=train, params=xgb_param, model_name=model_name, callbacks=[pruning_callback])
    elif model_name =='lgb':
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "validation_1-auc")
        mean_auc = cross_validate_tune_hyperparameters( train=train, params=lgb_param, model_name=model_name, callbacks=[pruning_callback])
    elif model_name == 'mlp':
        mean_auc = cross_validate_tune_hyperparameters( train=train, params=mlp_params, model_name=model_name)
    return mean_auc


In [52]:
all_trials = {}

In [72]:
pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(study_name='xgb',pruner=pruner, direction="maximize")
study.optimize(lambda trial: objective(trial, train=train, model_name='xgb'), n_trials=50)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
all_trials['xgb'] = trial.params.items()

[I 2024-05-26 17:17:40,606] A new study created in memory with name: xgb


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation_0-auc:0.82962	validation_1-auc:0.82910
[50]	validation_0-auc:0.88691	validation_1-auc:0.88655
[100]	validation_0-auc:0.89097	validation_1-auc:0.89060
[150]	validation_0-auc:0.89552	validation_1-auc:0.89508
[199]	validation_0-auc:0.89749	validation_1-auc:0.89701
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation_0-auc:0.82929	validation_1-auc:0.82976
[50]	validation_0-auc:0.88681	validation_1-auc:0.88682
[100]	validation_0-auc:0.89057	validation_1-auc:0.89050
[150]	validation_0-auc:0.89529	validation_1-auc:0.89529
[199]	validation_0-auc:0.89716	validation_1-auc:0.89717
===========fold 2================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation

[I 2024-05-26 17:18:09,279] Trial 0 finished with value: 0.8972004564287293 and parameters: {'learning_rate': 0.020203586153278344, 'max_depth': 2, 'colsample_bytree': 0.8, 'subsample': 1.0, 'lambda': 0.00020859458052474336, 'alpha': 3.989162019829639e-06, 'hidden_layer_sizes': 5}. Best is trial 0 with value: 0.8972004564287293.


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation_0-auc:0.76312	validation_1-auc:0.76305
[50]	validation_0-auc:0.88839	validation_1-auc:0.88796
[100]	validation_0-auc:0.89164	validation_1-auc:0.89129
[150]	validation_0-auc:0.89241	validation_1-auc:0.89206
[199]	validation_0-auc:0.89303	validation_1-auc:0.89263
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation_0-auc:0.76273	validation_1-auc:0.76382
[50]	validation_0-auc:0.88834	validation_1-auc:0.88815
[100]	validation_0-auc:0.89166	validation_1-auc:0.89141
[150]	validation_0-auc:0.89235	validation_1-auc:0.89211
[199]	validation_0-auc:0.89292	validation_1-auc:0.89268
===========fold 2================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation

[I 2024-05-26 17:18:35,639] Trial 1 finished with value: 0.8928140591704867 and parameters: {'learning_rate': 0.08831108534182419, 'max_depth': 1, 'colsample_bytree': 0.6, 'subsample': 1.0, 'lambda': 1.3429382570873412e-07, 'alpha': 0.08317671635416692, 'hidden_layer_sizes': 5}. Best is trial 0 with value: 0.8972004564287293.


...

===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation_0-auc:0.88852	validation_1-auc:0.88622


[I 2024-05-26 17:36:51,253] Trial 48 pruned. Trial was pruned at iteration 5.


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
XGBoost--------------
[0]	validation_0-auc:0.89691	validation_1-auc:0.89495


[I 2024-05-26 17:36:57,696] Trial 49 pruned. Trial was pruned at iteration 20.


Number of finished trials: 50
Best trial:
  Value: 0.9062830425363898
  Params: 
    learning_rate: 0.034630277480196384
    max_depth: 9
    colsample_bytree: 0.8
    subsample: 0.30000000000000004
    lambda: 1.3270228907353322e-06
    alpha: 0.0020136244579038245
    hidden_layer_sizes: 3


In [73]:
pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(study_name='lgb',pruner=pruner, direction="maximize")
study.optimize(lambda trial: objective(trial, train=train, model_name='lgb'), n_trials=50)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
all_trials['lgb'] = trial.params.items()

[I 2024-05-26 17:36:57,793] A new study created in memory with name: lgb


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
LGBModel--------------
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=1.239404753906364e-07. Current value: lambda_l2=1.239404753906364e-07
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=1.239404753906364e-07. Current value: lambda_l2=1.239404753906364e-07
[LightGBM] [Info] Number of positive: 162885, number of negative: 503781
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7208
[LightGBM] [Info] Number of data points in the train set: 666666, number of used features: 123
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] S

[I 2024-05-26 17:37:39,564] Trial 0 finished with value: 0.8754446994948714 and parameters: {'learning_rate': 0.05159575385157755, 'max_depth': 1, 'colsample_bytree': 0.1, 'subsample': 0.4, 'lambda': 1.239404753906364e-07, 'alpha': 2.681823137198342e-07, 'hidden_layer_sizes': 3}. Best is trial 0 with value: 0.8754446994948714.


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
LGBModel--------------
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=4.5401716432636326e-08. Current value: lambda_l2=4.5401716432636326e-08
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=4.5401716432636326e-08. Current value: lambda_l2=4.5401716432636326e-08
[LightGBM] [Info] Number of positive: 162885, number of negative: 503781
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7208
[LightGBM] [Info] Number of data points in the train set: 666666, number of used features: 123
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Inf

[I 2024-05-26 17:38:27,455] Trial 1 finished with value: 0.9005112448849425 and parameters: {'learning_rate': 0.07318806304212372, 'max_depth': 2, 'colsample_bytree': 0.30000000000000004, 'subsample': 0.9, 'lambda': 4.5401716432636326e-08, 'alpha': 3.734719921036922e-08, 'hidden_layer_sizes': 5}. Best is trial 1 with value: 0.9005112448849425.


...

===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
LGBModel--------------
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.00019999312851095498. Current value: lambda_l2=0.00019999312851095498
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.00019999312851095498. Current value: lambda_l2=0.00019999312851095498
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 162885, number of negative: 503781
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7208
[LightGBM] [Info] Number of data points in t

[I 2024-05-26 18:22:02,830] Trial 48 finished with value: 0.9052558080254807 and parameters: {'learning_rate': 0.02705225481126833, 'max_depth': 8, 'colsample_bytree': 0.9, 'subsample': 0.2, 'lambda': 0.00019999312851095498, 'alpha': 0.03994262151801138, 'hidden_layer_sizes': 2}. Best is trial 42 with value: 0.9058925272147199.


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
LGBModel--------------
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.9532901287692351. Current value: lambda_l2=0.9532901287692351
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.9532901287692351. Current value: lambda_l2=0.9532901287692351
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 162885, number of negative: 503781
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7208
[LightGBM] [Info] Number of data points in the train set: 66

[I 2024-05-26 18:23:00,697] Trial 49 finished with value: 0.9058059034247047 and parameters: {'learning_rate': 0.04811883709715961, 'max_depth': 9, 'colsample_bytree': 0.7000000000000001, 'subsample': 0.1, 'lambda': 0.9532901287692351, 'alpha': 5.961727545824861e-07, 'hidden_layer_sizes': 4}. Best is trial 42 with value: 0.9058925272147199.


Number of finished trials: 50
Best trial:
  Value: 0.9058925272147199
  Params: 
    learning_rate: 0.09249779656872704
    max_depth: 8
    colsample_bytree: 1.0
    subsample: 0.1
    lambda: 0.9648085464373999
    alpha: 0.9903394475964249
    hidden_layer_sizes: 4


In [55]:
pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(study_name='mlp',pruner=pruner, direction="maximize")
study.optimize(lambda trial: objective(trial, train=train, model_name='mlp'), n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
all_trials['mlp'] = trial.params.items()

[I 2024-05-26 11:07:01,386] A new study created in memory with name: mlp


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 2================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------


[I 2024-05-26 11:15:24,162] Trial 0 finished with value: 0.899397668877507 and parameters: {'learning_rate': 0.06293631247951315, 'max_depth': 4, 'colsample_bytree': 1.0, 'subsample': 0.2, 'lambda': 0.655164120003187, 'alpha': 0.00023388054209915076, 'hidden_layer_sizes': 2}. Best is trial 0 with value: 0.899397668877507.


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 2================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------


[I 2024-05-26 11:26:21,126] Trial 1 finished with value: 0.9011388708201604 and parameters: {'learning_rate': 0.010062311420025734, 'max_depth': 9, 'colsample_bytree': 0.7000000000000001, 'subsample': 0.8, 'lambda': 2.808309456046311e-05, 'alpha': 0.004971233814508131, 'hidden_layer_sizes': 3}. Best is trial 1 with value: 0.9011388708201604.


...

===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 2================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------


[I 2024-05-26 12:34:26,546] Trial 8 finished with value: 0.9011388708201604 and parameters: {'learning_rate': 0.06316039443312342, 'max_depth': 7, 'colsample_bytree': 0.4, 'subsample': 0.7000000000000001, 'lambda': 5.856084433639924e-08, 'alpha': 2.8607022055207455e-06, 'hidden_layer_sizes': 3}. Best is trial 2 with value: 0.9020526070928083.


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------
===========fold 2================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
MLP------------------


[I 2024-05-26 12:43:36,136] Trial 9 finished with value: 0.9020526070928083 and parameters: {'learning_rate': 0.08910810163144461, 'max_depth': 5, 'colsample_bytree': 0.4, 'subsample': 0.5, 'lambda': 0.23820714340564797, 'alpha': 5.482588542846957e-06, 'hidden_layer_sizes': 5}. Best is trial 2 with value: 0.9020526070928083.


Number of finished trials: 10
Best trial:
  Value: 0.9020526070928083
  Params: 
    learning_rate: 0.09489887655374188
    max_depth: 5
    colsample_bytree: 0.8
    subsample: 0.1
    lambda: 0.000231391542151867
    alpha: 0.6459845987300605
    hidden_layer_sizes: 5


In [56]:
pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(study_name='rf',pruner=pruner, direction="maximize")
study.optimize(lambda trial: objective(trial, train=train, model_name='rf'), n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
all_trials['rf'] = trial.params.items()

[I 2024-05-26 12:43:36,148] A new study created in memory with name: rf


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 
 best_threshold 0.3899999999999999 best_score 0.7823217623466623
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35

[I 2024-05-26 13:21:12,531] Trial 0 finished with value: 0.8989454023697793 and parameters: {'learning_rate': 0.018831971923188563, 'max_depth': 6, 'colsample_bytree': 1.0, 'subsample': 0.9, 'lambda': 2.7910822587618976e-08, 'alpha': 1.9404937284683135e-07, 'hidden_layer_sizes': 5}. Best is trial 0 with value: 0.8989454023697793.



 best_threshold 0.4099999999999998 best_score 0.7813584856597724
===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 
 best_threshold 0.3699999999999999 best_score 0.7575495505019685
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24

[I 2024-05-26 13:23:17,470] Trial 1 finished with value: 0.8699741711520782 and parameters: {'learning_rate': 0.01737306235084337, 'max_depth': 1, 'colsample_bytree': 0.2, 'subsample': 0.1, 'lambda': 8.915607778059581e-08, 'alpha': 2.672849991271086e-05, 'hidden_layer_sizes': 5}. Best is trial 0 with value: 0.8989454023697793.


...

===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 
 best_threshold 0.3699999999999999 best_score 0.7646882652541831
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35

[I 2024-05-26 16:22:21,254] Trial 8 finished with value: 0.8873885094856253 and parameters: {'learning_rate': 0.06175620628899644, 'max_depth': 4, 'colsample_bytree': 0.2, 'subsample': 0.5, 'lambda': 7.032313620107411e-07, 'alpha': 0.02130630658983126, 'hidden_layer_sizes': 2}. Best is trial 4 with value: 0.9035132069272324.


===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 
 best_threshold 0.3799999999999999 best_score 0.764260761290132
===========fold 1================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
Random Forest--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35,

[I 2024-05-26 16:27:15,478] Trial 9 finished with value: 0.8869749680246688 and parameters: {'learning_rate': 0.09008829942733483, 'max_depth': 6, 'colsample_bytree': 0.1, 'subsample': 1.0, 'lambda': 6.2181601318806825e-06, 'alpha': 1.976675428290502e-06, 'hidden_layer_sizes': 4}. Best is trial 4 with value: 0.9035132069272324.


0.89, 
 best_threshold 0.3599999999999999 best_score 0.7641913373592009
Number of finished trials: 10
Best trial:
  Value: 0.9035132069272324
  Params: 
    learning_rate: 0.060492102527360606
    max_depth: 9
    colsample_bytree: 0.5
    subsample: 0.2
    lambda: 0.12619375123209245
    alpha: 2.3009010377609224e-07
    hidden_layer_sizes: 2


In [81]:
lgb_items = {}
xgb_items = {}
rf_items = {}
mlp_items = {}
for k,v in all_trials['lgb']:
    lgb_items[k] = v
for k,v in all_trials['xgb']:
    xgb_items[k] = v
for k,v in all_trials['rf']:
    rf_items[k] = v
for k,v in all_trials['mlp']:
    mlp_items[k] = v

In [65]:
all_trials['xgb']

dict_items([('learning_rate', 0.09500314347224413), ('max_depth', 7), ('colsample_bytree', 0.2), ('subsample', 0.9), ('lambda', 0.027332136849109373), ('alpha', 0.0006337507915634194), ('hidden_layer_sizes', 2)])

In [66]:
all_trials['rf']

dict_items([('learning_rate', 0.060492102527360606), ('max_depth', 9), ('colsample_bytree', 0.5), ('subsample', 0.2), ('lambda', 0.12619375123209245), ('alpha', 2.3009010377609224e-07), ('hidden_layer_sizes', 2)])

In [83]:
lgb_items

{'learning_rate': 0.09249779656872704,
 'max_depth': 8,
 'colsample_bytree': 1.0,
 'subsample': 0.1,
 'lambda': 0.9648085464373999,
 'alpha': 0.9903394475964249,
 'hidden_layer_sizes': 4}

In [84]:
xgb_items

{'learning_rate': 0.034630277480196384,
 'max_depth': 9,
 'colsample_bytree': 0.8,
 'subsample': 0.30000000000000004,
 'lambda': 1.3270228907353322e-06,
 'alpha': 0.0020136244579038245,
 'hidden_layer_sizes': 3}

In [85]:
rf_items

{'learning_rate': 0.060492102527360606,
 'max_depth': 9,
 'colsample_bytree': 0.5,
 'subsample': 0.2,
 'lambda': 0.12619375123209245,
 'alpha': 2.3009010377609224e-07,
 'hidden_layer_sizes': 2}

In [86]:
N_ESTIMATORS = 200
SEED=42
LGBM_Hyperparameters =  {
    "n_estimators": N_ESTIMATORS,
    'learning_rate':lgb_items['learning_rate'],
     'max_depth':lgb_items['max_depth'],
     'colsample_bytree':lgb_items['colsample_bytree'],
     'subsample':lgb_items['subsample'],
     'alpha':lgb_items['alpha'],
     'lambda':lgb_items['lambda'],

    "random_state":SEED,
    'device':'gpu',
#     "class_weight": "balanced"
}

XGBoost_Hyperparameters = {
    'objective' : 'binary:logistic',
     'eval_metric':['logloss', 'auc'],
     'n_estimators':N_ESTIMATORS,
     'learning_rate': xgb_items['learning_rate'],
     'max_depth':int(xgb_items['max_depth']),
     'colsample_bytree': xgb_items['colsample_bytree'],
     'subsample':xgb_items['subsample'],
     'lambda': xgb_items['lambda'],
     'alpha':xgb_items['alpha'],
     'seed':SEED,
     # 'scale_pos_weight':3,
     'enable_categorical':True,
     'early_stopping_rounds': 50,
     'tree_method':'gpu_hist'}
RF_Hyperparameters = {
    'n_estimators': N_ESTIMATORS,
    'max_depth':rf_items['max_depth'],
    'random_state':SEED,
    'max_features': rf_items['colsample_bytree'], 
    'n_jobs': -1
}
Logreg_Hyperparameters = {'max_iter':N_ESTIMATORS,'random_state':SEED} 
MLP_Hyperparametesr = {'hidden_layer_sizes':(mlp_items['hidden_layer_sizes'],125), 'random_state':SEED, 'max_iter':min(N_ESTIMATORS,100)}
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

def cross_validate(train, USE_SMOTE=False, USE_CLASS_WEIGHT=False, USE_UNDER_SAMPLING=False):
    oofs = np.zeros((train.shape[0], len(MODEL_NAMES)))
    for i, (train_index, valid_index) in enumerate(kfold.split(train, train[TARGET])):
        print(f"===========fold {i}================")
        X_train, oh_encoder, robust_scaler = process_data(train.iloc[train_index])
        X_valid, _, _  = process_data(train.iloc[valid_index], oh_encoder,robust_scaler)
        print(X_train.isnull().sum())
        y_train = train.iloc[train_index][TARGET].values
        y_valid = train.iloc[valid_index][TARGET].values
        logreg_hyperparameters = Logreg_Hyperparameters.copy()
        lgb_hyperparameters = LGBM_Hyperparameters.copy()
        xgboost_hyperparameters = XGBoost_Hyperparameters.copy()
        rf_hyperparameters = RF_Hyperparameters.copy()
        mlp_hyperparameters = MLP_Hyperparametesr.copy()
        if USE_SMOTE:
            print("SMOTEEEE")
            sm = SMOTE(random_state=42)
            X_train, y_train = sm.fit_resample(X_train, y_train)
        elif USE_CLASS_WEIGHT:
            print("CLASS_WEIGHTTTT")
           
            class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=y_train)
            class_weights =  {0: class_weights[0], 1: class_weights[1]}
            lgb_hyperparameters['class_weight'] = class_weights
            logreg_hyperparameters['class_weight'] = class_weights
            xgboost_hyperparameters['scale_pos_weight'] = class_weights[1] /  class_weights[0]
            rf_hyperparameters['class_weight'] = class_weights
        elif USE_UNDER_SAMPLING:
            print("UNDER SAMPLING")
            rus = RandomUnderSampler(random_state=42)
            X_train, y_train = rus.fit_resample(X_train, y_train)
        print("LOGREG--------------")
        logreg_model = LogisticRegression(**logreg_hyperparameters)
        logreg_model.fit(X_train, y_train)
        logreg_y_pred_proba = logreg_model.predict_proba(X_valid)[:,1]
        best_threshold, best_score = find_best_threshold_f1_score(y_valid, logreg_y_pred_proba)
        print(f"\n best_threshold {best_threshold} best_score {best_score}")
        y_pred = [1 if y_hat >=best_threshold else 0 for y_hat in logreg_y_pred_proba]

        print(roc_auc_score(y_valid, logreg_y_pred_proba))

        print(classification_report(y_valid, y_pred, digits=4))
        oofs[valid_index,0] = logreg_y_pred_proba

        print("Random Forest--------------")
        rf_model = RandomForestClassifier(**rf_hyperparameters)
        rf_model.fit(X_train, y_train)
        rf_y_pred_proba = rf_model.predict_proba(X_valid)[:,1]
        best_threshold, best_score = find_best_threshold_f1_score(y_valid, rf_y_pred_proba)
        print(f"\n best_threshold {best_threshold} best_score {best_score}")
        y_pred = [1 if y_hat >=best_threshold else 0 for y_hat in rf_y_pred_proba]

        print(roc_auc_score(y_valid, rf_y_pred_proba))

        print(classification_report(y_valid, y_pred, digits=4))
        oofs[valid_index,1] = rf_y_pred_proba
    #     models.append(model)
        print("LGBModel--------------")
        lgb_model = LGBMClassifier(**lgb_hyperparameters)
        callbacks = [lgb.early_stopping(200, verbose=50), lgb.log_evaluation(period=50)]
        lgb_model.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  eval_metric=["logloss", "auc"],
                  callbacks=callbacks)

        lgb_y_pred_proba = lgb_model.predict_proba(X_valid)[:,1]
        best_threshold, best_score = find_best_threshold_f1_score(y_valid, lgb_y_pred_proba)
        print(f"\n best_threshold {best_threshold} best_score {best_score}")
        y_pred = [1 if y_hat >=best_threshold else 0 for y_hat in lgb_y_pred_proba]
        print(roc_auc_score(y_valid, lgb_y_pred_proba))
        print(classification_report(y_valid, y_pred, digits=4))
        oofs[valid_index,2] = lgb_y_pred_proba
    #     models.append(model)
        # display(pd.DataFrame({'score': lgb_model.feature_importances_, 'feature': lgb_model.feature_name_}).sort_values('score',ascending=False))

        print("XGBoost--------------")
        xgb_model = XGBClassifier(**xgboost_hyperparameters)
        xgb_model.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=50)
        xgb_y_pred_proba = xgb_model.predict_proba(X_valid)[:,1]
        best_threshold, best_score = find_best_threshold_f1_score(y_valid, xgb_y_pred_proba)
        print(f"\n best_threshold {best_threshold} best_score {best_score}")
        y_pred = [1 if y_hat >=best_threshold else 0 for y_hat in xgb_y_pred_proba]
        print(roc_auc_score(y_valid, xgb_y_pred_proba))
        print(classification_report(y_valid, y_pred, digits=4))
        oofs[valid_index, 3] = xgb_y_pred_proba
        
        print("MLP------------------")
        mlp_model = MLPClassifier(**mlp_hyperparameters)
        mlp_model.fit(X_train, y_train)
        mlp_y_pred_proba = mlp_model.predict_proba(X_valid)[:,1]
        best_threshold, best_score = find_best_threshold_f1_score(y_valid, mlp_y_pred_proba)
        print(f"\n best_threshold {best_threshold} best_score {best_score}")
        y_pred = [1 if y_hat >=best_threshold else 0 for y_hat in mlp_y_pred_proba]
        print(roc_auc_score(y_valid, mlp_y_pred_proba))
        print(classification_report(y_valid, y_pred, digits=4))
        oofs[valid_index, 4] = mlp_y_pred_proba
    return oofs

In [87]:
accuracy_scores = []
f1_scores = []
auc_scores = []
specificity_scores = []
sensitivity_scores = []
def scoring(y_test,y_pred_proba, best_threshold):
    y_pred = [1 if y_hat >= best_threshold else 0 for y_hat in y_pred_proba]
    acc = accuracy_score(y_test, y_pred)
    _f1_score = f1_score(y_test, y_pred, average='macro')
    auc_score = roc_auc_score(y_test, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    print("accuracy", acc)
    print("f1_score", _f1_score)
    print("auc", auc_score)
    print("sensitivity", sensitivity, "specificity", specificity)
    print(classification_report(y_test, y_pred, digits=4))
    return acc, _f1_score, auc_score, specificity, sensitivity

In [88]:
def train_model(oofs,X_train, y_train, X_test, y_test, USE_SMOTE=False,USE_CLASS_WEIGHT=False, USE_UNDER_SAMPLING=False):
    models = []
    predictions = []
    accuracy_scores = []
    f1_scores = []
    auc_scores = []
    specificity_scores = []
    sensitivity_scores = []
    best_thresholds = []
    for i in range(len(MODEL_NAMES)):
        best_threshold, best_score = find_best_threshold_f1_score(train[TARGET].values, oofs[:,i])
        best_thresholds.append(best_threshold)
        print('\n',best_threshold, best_score)
    logreg_hyperparameters = Logreg_Hyperparameters.copy()
    lgb_hyperparameters = LGBM_Hyperparameters.copy()
    xgboost_hyperparameters = XGBoost_Hyperparameters.copy()
    del xgboost_hyperparameters['early_stopping_rounds']
    rf_hyperparameters = RF_Hyperparameters.copy()
    mlp_hyperparameters = MLP_Hyperparametesr.copy()
    if USE_SMOTE:
        print("SMOTEEEE")
        sm = SMOTE(random_state=42)
        X_train, y_train = sm.fit_resample(X_train, y_train)
    elif USE_CLASS_WEIGHT:
        print("CLASS_WEIGHTTTT")
        class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=y_train)
        class_weights =  {0: class_weights[0], 1: class_weights[1]}
        lgb_hyperparameters['class_weight'] = class_weights
        logreg_hyperparameters['class_weight'] = class_weights
        xgboost_hyperparameters['scale_pos_weight'] = class_weights[1]/ class_weights[0]
        rf_hyperparameters['class_weight'] = class_weights
    elif USE_UNDER_SAMPLING:
        print("UNDER SAMPLING")
        rus = RandomUnderSampler(random_state=42)
        X_train, y_train = rus.fit_resample(X_train, y_train)

    print("LOGREG--------------")
    logreg_model = LogisticRegression(**logreg_hyperparameters)
    logreg_model.fit(X_train, y_train)
    logreg_y_pred_proba = logreg_model.predict_proba(X_test)[:,1]
    acc, _f1_score, auc_score, specificity, sensitivity = scoring(y_test,logreg_y_pred_proba,best_thresholds[0])
    accuracy_scores.append(acc)
    f1_scores.append(_f1_score)
    auc_scores.append(auc_score)
    specificity_scores.append(specificity)
    sensitivity_scores.append(sensitivity)
    predictions.append(logreg_y_pred_proba)
    models.append(logreg_model)

    print("Random Forest--------------")
    rf_model = RandomForestClassifier(**rf_hyperparameters)
    rf_model.fit(X_train, y_train)
    rf_y_pred_proba = rf_model.predict_proba(X_test)[:,1]
    acc, _f1_score, auc_score, specificity, sensitivity = scoring(y_test,rf_y_pred_proba,best_thresholds[1])
    accuracy_scores.append(acc)
    f1_scores.append(_f1_score)
    auc_scores.append(auc_score)
    specificity_scores.append(specificity)
    sensitivity_scores.append(sensitivity)
    predictions.append(rf_y_pred_proba)
    models.append(rf_model)

    print("LGBModel--------------")
    lgb_model = LGBMClassifier(**lgb_hyperparameters)
    lgb_model.fit(X_train, y_train)
    lgb_y_pred_proba = lgb_model.predict_proba(X_test)[:,1]
    acc, _f1_score, auc_score, specificity, sensitivity = scoring(y_test,lgb_y_pred_proba,best_thresholds[2])
    accuracy_scores.append(acc)
    f1_scores.append(_f1_score)
    auc_scores.append(auc_score)
    specificity_scores.append(specificity)
    sensitivity_scores.append(sensitivity)
    predictions.append(lgb_y_pred_proba)
    models.append(lgb_model)

    print("XGBoost--------------")
    print(xgboost_hyperparameters)
    xgb_model = XGBClassifier(**xgboost_hyperparameters)
    xgb_model.fit(X_train, y_train)
    xgb_y_pred_proba = xgb_model.predict_proba(X_test)[:,1]
    acc, _f1_score, auc_score, specificity, sensitivity = scoring(y_test,xgb_y_pred_proba,best_thresholds[3])
    accuracy_scores.append(acc)
    f1_scores.append(_f1_score)
    auc_scores.append(auc_score)
    specificity_scores.append(specificity)
    sensitivity_scores.append(sensitivity)
    predictions.append(xgb_y_pred_proba)
    models.append(xgb_model)
    
    print("MLP--------------")
    mlp_model = MLPClassifier(**mlp_hyperparameters)
    mlp_model.fit(X_train, y_train)
    mlp_y_pred_proba = mlp_model.predict_proba(X_test)[:,1]
    acc, _f1_score, auc_score, specificity, sensitivity = scoring(y_test,mlp_y_pred_proba,best_thresholds[4])
    accuracy_scores.append(acc)
    f1_scores.append(_f1_score)
    auc_scores.append(auc_score)
    specificity_scores.append(specificity)
    sensitivity_scores.append(sensitivity)
    predictions.append(mlp_y_pred_proba)
    models.append(mlp_model)

    print(MODEL_NAMES)
    print(accuracy_scores)
    print(f1_scores)
    print(auc_scores)
    print(specificity_scores)
    print(sensitivity_scores)
    score_df = pd.DataFrame({'model_name': MODEL_NAMES,
                         'accuracy_score':accuracy_scores, 
                         'f1_score': f1_scores, 
                         'auc_score': auc_scores, 
                         'specificity_score': specificity_scores, 
                         'sensitivity_score': sensitivity_scores})
    return score_df,models, predictions

In [89]:
X_train,oh_encoder,robust_scaler = process_data(train)
X_test, _,_ = process_data(test,oh_encoder,robust_scaler)
y_train = train[TARGET].values
y_test = test[TARGET].values

fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler


In [90]:
print("Not handle Balanced")
nothing_oofs = cross_validate(train)
nothing_score_df, nothing_models, nothing_predictions = train_model(nothing_oofs,X_train, y_train, X_test, y_test)

Not handle Balanced
===========fold 0================
fit train OneHotEncoder
fit train RobustScaler
loadd onehot encoder
loadd robust scaler
os_name_macos          0
os_name_windows        0
age_group_15-17        0
age_group_18-24        0
age_group_25-34        0
                      ..
work_count             0
social_count           0
news_count             0
entertainment_count    0
ecommerce_count        0
Length: 123, dtype: int64
LOGREG--------------
0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 
 best_threshold 0.4199999999999998 best_score 0.7654678

In [91]:
all_trials

{'xgb': dict_items([('learning_rate', 0.034630277480196384), ('max_depth', 9), ('colsample_bytree', 0.8), ('subsample', 0.30000000000000004), ('lambda', 1.3270228907353322e-06), ('alpha', 0.0020136244579038245), ('hidden_layer_sizes', 3)]),
 'lgb': dict_items([('learning_rate', 0.09249779656872704), ('max_depth', 8), ('colsample_bytree', 1.0), ('subsample', 0.1), ('lambda', 0.9648085464373999), ('alpha', 0.9903394475964249), ('hidden_layer_sizes', 4)]),
 'mlp': dict_items([('learning_rate', 0.09489887655374188), ('max_depth', 5), ('colsample_bytree', 0.8), ('subsample', 0.1), ('lambda', 0.000231391542151867), ('alpha', 0.6459845987300605), ('hidden_layer_sizes', 5)]),
 'rf': dict_items([('learning_rate', 0.060492102527360606), ('max_depth', 9), ('colsample_bytree', 0.5), ('subsample', 0.2), ('lambda', 0.12619375123209245), ('alpha', 2.3009010377609224e-07), ('hidden_layer_sizes', 2)])}

In [92]:
nothing_score_df

,model_name,accuracy_score,f1_score,auc_score,specificity_score,sensitivity_score
0,log_reg,0.821340,0.766071,0.882068,0.865410,0.685259
1,randomforest,0.836955,0.785171,0.903207,0.878986,0.707170
2,lightgbm,0.836160,0.787486,0.905551,0.870262,0.730859
3,xgboost,0.837880,0.787703,0.905978,0.876418,0.718882
4,mlp,0.834835,0.784627,0.903063,0.872195,0.719474


In [93]:
import pickle
with open(f"../checkpoints/hyper_nothing_model.pkl", "wb") as f:
    pickle.dump({
            "score_df":nothing_score_df,
            "oofs": nothing_oofs,
            "models": nothing_models,
            "model_names": ['lr', 'rf', 'lgb', 'xgb', 'mlp'],
            "predictions":nothing_predictions},f,protocol=pickle.HIGHEST_PROTOCOL)